<a href="https://colab.research.google.com/github/sahilfaizal01/Quantitative-Finance-Projects/blob/main/Predict_FACEBOOK_Stock_Prices_Using_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
!pip install quandl

In [31]:
import quandl
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split

In [32]:
df = quandl.get("WIKI/FB")

In [33]:
df.head()

,Open,High,Low,Close,Volume,Ex-Dividend,Split Ratio,Adj. Open,Adj. High,Adj. Low,Adj. Close,Adj. Volume
Date,,,,,,,,,,,,
2012-05-18,42.05,45.00,38.00,38.2318,573576400.0,0.0,1.0,42.05,45.00,38.00,38.2318,573576400.0
2012-05-21,36.53,36.66,33.00,34.0300,168192700.0,0.0,1.0,36.53,36.66,33.00,34.0300,168192700.0
2012-05-22,32.61,33.59,30.94,31.0000,101786600.0,0.0,1.0,32.61,33.59,30.94,31.0000,101786600.0
2012-05-23,31.37,32.50,31.36,32.0000,73600000.0,0.0,1.0,31.37,32.50,31.36,32.0000,73600000.0
2012-05-24,32.95,33.21,31.77,33.0300,50237200.0,0.0,1.0,32.95,33.21,31.77,33.0300,50237200.0


In [34]:
df.columns

Index(['Open', 'High', 'Low', 'Close', 'Volume', 'Ex-Dividend', 'Split Ratio',
       'Adj. Open', 'Adj. High', 'Adj. Low', 'Adj. Close', 'Adj. Volume'],
      dtype='object')

In [35]:
df.shape

(1472, 12)

In [36]:
# get the adjusted close price
df_n = df[['Adj. Close']]
df_n.head()

,Adj. Close
Date,
2012-05-18,38.2318
2012-05-21,34.0300
2012-05-22,31.0000
2012-05-23,32.0000
2012-05-24,33.0300


In [37]:
# a variable for predicting 'n' days into the future
forecast_out = 30
# create another column (target variable) shifted 'n' units up
df_n['Prediction'] = df_n[['Adj. Close']].shift(-forecast_out)
df_n.head()

<ipython-input-37-a3714ccff9f4>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_n['Prediction'] = df_n[['Adj. Close']].shift(-forecast_out)


,Adj. Close,Prediction
Date,,
2012-05-18,38.2318,30.771
2012-05-21,34.0300,31.200
2012-05-22,31.0000,31.470
2012-05-23,32.0000,31.730
2012-05-24,33.0300,32.170


In [38]:
df_n.tail()

,Adj. Close,Prediction
Date,,
2018-03-21,169.39,NaN
2018-03-22,164.89,NaN
2018-03-23,159.39,NaN
2018-03-26,160.06,NaN
2018-03-27,152.19,NaN


In [39]:
# creating independent dataset (X)
X = np.array(df_n.drop(['Prediction'],1))
# remove the last n rows
X = X[:-forecast_out]
print(X)

[[ 38.2318]
 [ 34.03  ]
 [ 31.    ]
 ...
 [171.5499]
 [175.98  ]
 [176.41  ]]


<ipython-input-39-95bf066e5b2c>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = np.array(df_n.drop(['Prediction'],1))


In [40]:
# create dependent variable (y) - all values including NaN's
y = np.array(df_n['Prediction'])
# get all y values except last n rows
y = y[:-forecast_out]
print(y)

[ 30.771  31.2    31.47  ... 159.39  160.06  152.19 ]


In [41]:
# split the data into 80% training and 20% testing
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [42]:
# create and train the model
# SVM (Regressor)
svr_rbf = SVR(kernel='rbf',C=1e3,gamma=0.1)
svr_rbf.fit(X_train,y_train)

SVR(C=1000.0, gamma=0.1)

In [43]:
# r2 determination
svm_conf = svr_rbf.score(X_test,y_test)
print('SVM Confidence:',svm_conf)

SVM Confidence: 0.9842393033683633


In [44]:
# Linear Regression MODEL
lr = LinearRegression()
lr.fit(X_train,y_train)
lr_conf = lr.score(X_test,y_test)
print('LR Confidence:',lr_conf)

LR Confidence: 0.980681510192342


In [45]:
x_forecast = np.array(df_n.drop(['Prediction'],1))[-forecast_out:]
print(x_forecast)

[[173.15]
 [179.52]
 [179.96]
 [177.36]
 [176.01]
 [177.91]
 [178.99]
 [183.29]
 [184.93]
 [181.46]
 [178.32]
 [175.94]
 [176.62]
 [180.4 ]
 [179.78]
 [183.71]
 [182.34]
 [185.23]
 [184.76]
 [181.88]
 [184.19]
 [183.86]
 [185.09]
 [172.56]
 [168.15]
 [169.39]
 [164.89]
 [159.39]
 [160.06]
 [152.19]]


<ipython-input-45-6cf412236b3c>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  x_forecast = np.array(df_n.drop(['Prediction'],1))[-forecast_out:]


In [46]:
# print the predictions for next n days
lr_predictions = lr.predict(x_forecast)
print(lr_predictions)

[177.14828388 183.58159488 184.02596801 181.40012678 180.03670923
 181.9555932  183.04632724 187.38906465 189.04536449 185.5408764
 182.36966816 179.9660135  180.65277198 184.47034114 183.844179
 187.813239   186.42962266 189.34834617 188.87367488 185.96505075
 188.29800968 187.96472984 189.20695472 176.55241991 172.09858923
 173.3509135  168.80618831 163.25152419 163.92818327 155.97996389]


In [47]:
svm_predictions = svr_rbf.predict(x_forecast)
print(svm_predictions)

[174.88501822 181.5829651  181.96927841 175.62618799 174.56699488
 176.95091492 180.52487934 187.02917848 179.0475968  184.50556146
 178.33031638 174.49715244 174.93782713 182.23641752 181.84856455
 185.08551119 187.49840536 178.69284817 179.52978    186.05627449
 182.27970628 184.23286945 178.77729299 176.97359342 171.46436845
 172.20793703 172.28432088 167.65610457 166.75035714 158.68755474]
